In [54]:
# Import required libraries
import tabula
import pandas as pd
import os
import json

In [ ]:
# Ensure you have JAVA installed
!java -version

## Introduction

This script will complete two main functionalities:

1. Extract all transactions from a directory containing RBC Visa statements (as PDF files) into a CSV file.
2. Assign all transactions categories based on the classifications specified in the `transaction_classification.json` JSON file.

**Requirements: File Naming Convention:**

1. Each PDF file **must** end with a **hyphen** followed by the **year-month-day** of the statement in format `YYYY-MM-DD`. For example, a September 2022 statement filename must end in the following format: "... -2022-09".
2. You must specify the source and destination directory in the code chunk below.

In [96]:
SOURCE_DIR = r'./statements' # statements location
RESULTS_DIR = r'./results' # output file location
CATEGORY_FILE = r'transaction_classification.json' # category classification file

## Helper Functions

In [97]:
# Define a function to clean up the format of the price entries
def parse_price(price):
    # Define a factor variable to multiply the price with (for debit/credit)
    factor = 1
    
    # Remove all commas in the price string
    price = price.replace(",", "")
    
    # If the price has a negative sign, it is a payment
    if price[0] == '-':
        factor = -1
        price = price[1:]
        
    # Convert the string to a float and multiply by appropriate factor
    return float(price[1:]) * factor

## Main Code

In [98]:
# Define valid months as they appear in the RBC statements
valid_months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
                'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Create a master dataframe to store all extracted transactions from all statements
master_df = pd.DataFrame(columns = ['Transaction Date', 'Posting Date', 'Activity', 'Amount (CAD)'])

In [99]:
# Iterate over each statement PDF
res = pd.DataFrame()
for entry in os.scandir(SOURCE_DIR):
    
    # Ensure that the file is a PDF
    if entry.name[-4:] == '.pdf':
    
        # Parse statement based on identified area markers
        # If the statement layout changes, this will likely break and need to be updated
        pdf_path = entry
        dfs = tabula.read_pdf(pdf_path, 
                              stream=True,
                              pages='all', 
                              pandas_options={'header': None}, 
                              area=(195, 57, 800, 353),
                              columns = [58, 95.8, 128, 301]) # Columns is very important!

        # Create a dataframe to store all transactions from all pages for this PDF

        # Combine transactions from all pages (if > 1 page) and clean column names
        df_transactions = pd.concat(dfs)
        df_transactions = df_transactions.drop(df_transactions.columns[0], axis='columns').dropna()
        df_transactions.index = np.arange(0, len(df_transactions))
        df_transactions.columns = ['Transaction Date', 'Posting Date', 'Activity', 'Amount (CAD)']
        
        # Drop all rows that don't include transactions
        df_transactions = df_transactions.loc[df_transactions['Transaction Date'].str.startswith(tuple(valid_months))]
        
        # Clean up the transaction amount column values
        df_transactions['Amount (CAD)'] = df_transactions['Amount (CAD)'].apply(lambda x: parse_price(x))
        
        # Assign the year to the date
        # This involves custom logic for Dec/Jan statements with transactions that span multiple years
        statement_year_month = entry.name[-14:-7]
        statement_year = statement_year_month.split('-')[0]
        statement_month = statement_year_month.split('-')[1]
        
        # If the month is not Dec/Jan, assign simple year to transaction
        if statement_month not in ['01', '12']:
            df_transactions['Transaction Date'] = df_transactions['Transaction Date'] + " " + statement_year
            df_transactions['Posting Date'] = df_transactions['Posting Date'] + " " + statement_year
        
        # If the statement is for January and contains (STATEMENT_YEAR - 1) transactions for December, we need to specify those
        elif statement_month in ['01']:
            df_transactions['Transaction Date'] = df_transactions.apply(lambda x: x['Transaction Date'] + " {}".format(int(statement_year)-1) \
                                                                        if x['Transaction Date'][:3] == "DEC" \
                                                                        else x['Transaction Date'] + " {}".format(int(statement_year)), axis='columns')
            
            df_transactions['Posting Date'] = df_transactions.apply(lambda x: x['Posting Date'] + " {}".format(int(statement_year)-1) \
                                                                        if x['Posting Date'][:3] == "DEC" \
                                                                        else x['Posting Date'] + " {}".format(int(statement_year)), axis='columns')
            
        # If the statement is for December and contains (STATEMENT_YEAR + 1) transactions for January, we need to specify those
        # This should not be the case with the current statement format from RBC, but just to catch edge cases we include this here
        elif statement_month in ['12']:
            df_transactions['Transaction Date'] = df_transactions.apply(lambda x: x['Transaction Date'] + " {}".format(int(statement_year)+1) \
                                                                        if x['Transaction Date'][:3] == "JAN" \
                                                                        else x['Transaction Date'] + " {}".format(int(statement_year)), axis='columns')
            
            df_transactions['Posting Date'] = df_transactions.apply(lambda x: x['Posting Date'] + " {}".format(int(statement_year)+1) \
                                                                    if x['Posting Date'][:3] == "JAN" \
                                                                    else x['Posting Date'] + " {}".format(int(statement_year)), axis='columns')
        
        # Convert all time columns to DATETIME format
        df_transactions['Transaction Date'] = pd.to_datetime(df_transactions['Transaction Date'],format='%b %d %Y').dt.date
        df_transactions['Posting Date'] = pd.to_datetime(df_transactions['Posting Date'],format='%b %d %Y').dt.date
        
        # Append the transactions to the master dataframe
        master_df = master_df.append(df_transactions)
    
# Sort the final dataframe by transaction dates
master_df = master_df.sort_values('Transaction Date')

# Re-index the rows
master_df.index = np.arange(0, len(master_df))

## Assigning Categories

Now we utilize the categories defined in the `transaction_classification.json` file to assign each activity a category.

In [100]:
# Read in JSON file
categories_file = CATEGORY_FILE
if CATEGORY_FILE != "":
    with open(categories_file) as json_file: 
        categories = json.load(json_file)

    # Create new column for category
    master_df['Category'] = None

    # Iterate through categories and assign the transaction the category based on regex matching
    for category in categories:
        for value in categories[category]:
            master_df.loc[master_df['Activity'].str.contains(value, regex=False), 'Category'] = category

## Write Results to CSV File

Now we export the parsed transactions as a CSV file.

Naming format: `statements_parsed_{min_txn_date}_{max_txn_date}.csv`

The `min_txn_date` is the minimum transaction date parsed from the statements. The `max_txn_date` is the maximum transaction date parsed from the statements. 

In [101]:
min_txn_date = master_df['Transaction Date'].min().strftime('%Y%m%d')
max_txn_date = master_df['Transaction Date'].max().strftime('%Y%m%d')

# Write to CSV
master_df.to_csv('{}/statements_parsed_{}_{}.csv'.format(RESULTS_DIR, min_txn_date, max_txn_date), index=False)